<a href="https://colab.research.google.com/github/raqgmar/tsa4dst/blob/main/06_01_INTERPRETABILIDAD_SVR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0 Preparación del entorno.

## 0.1 Definición de parámetros

In [141]:
# tfm_path='C:/Users/raqga/OneDrive - Universidad Complutense de Madrid (UCM)/Documentos/tsa4dst/TFM_data/'
tfm_path = '/content/drive/MyDrive/TFM data/'
H1_code = 'OMNI2_H0_MRG1HR'
lookback = 12
tfm_path_Nh_models = f'PRED_{lookforward}h/'
cols_to_use = ['Bx', 'By_gse', 'Bz_gse', 'By_gsm', 'Bz_gsm', 'P_density', 'E_field', 'plasma_T', 'plasma_V', 'Dst'] # 'AP', out
col_to_pred = "Dst"
hstorms_data = 'historical_storms_gruet2018.csv'
weak_threshold = -30 #1
moderate_threshold = -50 #2
strong_threshold = -100 #3
severe_threshold = -200 #4
great_threshold = -300 #5
gamma_value=0.0001
temporal_margin=5*24 # margen para obtener tiempos ampliados de las tormentas de gruet et al 2018
test_size = 0.2
kernels = ['linear', 'poly', 'rbf', 'sigmoid']

In [142]:
PATH_NN_RES_BY_STORM = "SVR_{}h_res_by_storm/storm_{}.csv"
PATH_NN_RES_BY_STORM_PLOTS = "SVR_{}h_res_by_storm/storm_{}.png"

## 0.2 Montar Google Drive (obtención de datos)

In [143]:
from google.colab import drive
drive.mount('/content/drive')
!rm -rf sample_data/

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 0.3 Importación de librerías

In [144]:
import os

import numpy as np

# librerías de manipulación de datos y gráficos
import pandas as pd
import matplotlib.dates as mdates
import numpy as np

# rfecv
from sklearn.feature_selection import RFECV
from sklearn.pipeline import Pipeline

# Función para desescalar los

# gráficos
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objs as go
from plotly.subplots import make_subplots

# modelo
from sklearn.svm import SVR
#from thundersvm import SVR
# escalado y división en train/test
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# obtención de métricas
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.metrics import mean_squared_log_error, median_absolute_error
from sklearn.metrics import explained_variance_score, max_error

# meta
import time
from math import sqrt


## 0.4 Definición de funciones

In [145]:

def imputar_nan(df):
  df.interpolate(method='linear', inplace=True)
  df.fillna(method='ffill', inplace=True)
  df.fillna(method='bfill', inplace=True)
  if sum(df.isnull().sum())!=0:
    print("Faltan nulos por tratar")
  return df


def filter_storms(df, historical_storms, temporal_margin):
    """
    Filter DataFrame entries based on the occurrence of storms within specific time intervals.

    Parameters:
    df (pandas.DataFrame): DataFrame containing time-series data with a 'Datetime' column.
    historical_storms (pandas.DataFrame): DataFrame containing the start and end times of historical storms.
    temporal_margin (int): Number of rows before and after the minimum Dst index to include in the result.

    Returns:
    list: A list of DataFrame snippets corresponding to the specified storm intervals.
    """
    all_storms = []
    for i in range(len(historical_storms)):
        df_tmp = df[(df["Datetime"] >= historical_storms.iloc[i]["start"]) & (df["Datetime"] <= historical_storms.iloc[i]["end"])]
        idx = df_tmp['Dst'].idxmin()
        all_storms.append(df.iloc[idx-temporal_margin:idx+temporal_margin])
    return all_storms

def combinar_dataframes_solapados(dfs):
    """
    Combines overlapping DataFrames in a list into non-overlapping DataFrames based on the 'Datetime' column.

    Parameters:
    dfs (list of pandas.DataFrame): List of DataFrames to combine.

    Returns:
    list: A list of combined DataFrames without overlap.
    """
    dfs.sort(key=lambda x: x['Datetime'].min())
    combinados = []
    combinacion_actual = dfs[0]

    for df in dfs[1:]:
        if df['Datetime'].min() <= combinacion_actual['Datetime'].max():
            combinacion_actual = pd.concat([combinacion_actual, df]).drop_duplicates().sort_values(by='Datetime')
        else:
            combinados.append(combinacion_actual)
            combinacion_actual = df
    combinados.append(combinacion_actual)
    return combinados

def scale_data(list_dfs, cols_to_use, col_to_pred):
    """
    Scales columns in a list of DataFrames using StandardScaler.

    Parameters:
    list_dfs (list of pandas.DataFrame): List of DataFrames to scale.
    cols_to_use (list of str): Column names to apply scaling to.
    col_to_pred (str): Column name used as a label for prediction.

    Returns:
    tuple: A tuple containing the list of scaled DataFrames and the label scaler.
    """
    list_dfs_ = []
    scaler_cols = StandardScaler()
    scaler_label = StandardScaler()
    scaler_cols.fit(pd.concat(list_dfs)[cols_to_use])
    scaler_label.fit(np.asarray(pd.concat(list_dfs)[col_to_pred]).reshape(-1,1))

    for df_ in list_dfs:
        df = df_.copy()
        df[cols_to_use] = scaler_cols.transform(df[cols_to_use])
        list_dfs_.append(df)

    return list_dfs_, scaler_label





Función para extraer las fechas de la nueva división de datos, una vez ampliadas las ventanas de cada tormenta a +-5 días desde el mínimo de cada tormenta, y eliminados los datos solapados

In [146]:
def get_new_splits_dates(all_storms):
    dates_start = []
    dates_end = []
    num_storm = []
    min_dst_values = []

    for idx, df in enumerate(all_storms):
        dates_start.append(df["Datetime"].iloc[0])
        dates_end.append(df["Datetime"].iloc[-1])
        num_storm.append(idx+1)
        min_dst_values.append(df["Dst"].min())
    df_new_storms = pd.DataFrame(
        {
            "storm_index": num_storm,
            "date_start": dates_start,
            "date_end": dates_end,
            "min_DST": min_dst_values
        }
    )
    return df_new_storms

Función para sacar los scalers de las variables de input y de la label (Dst)

In [147]:
def get_scalers(list_dfs, cols_to_use, col_to_pred):
    list_dfs_ = []
    scaler_cols = StandardScaler()
    scaler_label = StandardScaler()
    scaler_cols.fit(pd.concat(list_dfs)[cols_to_use])
    scaler_label.fit(np.asarray(pd.concat(list_dfs)[col_to_pred]).reshape(-1,1))

    return scaler_cols, scaler_label

Nueva función para crear ventanas de datos para input del modelo

In [148]:
def create_window_df_nn(list_dfs, lookback, lookforward, cols_to_use, col_to_pred, scaler_cols):
    x_train, y_train = [], []

    for df_ in list_dfs:
        df = df_[cols_to_use].copy()
        df.loc[:, cols_to_use] = scaler_cols.transform(df[cols_to_use])

        for i in range(len(df) - lookback - lookforward + 1):
            x_train.append(np.asarray(df.iloc[i:i+lookback].values))
            y_train.append(np.asarray(df.iloc[i+lookback+lookforward-1][col_to_pred]))

    return np.asarray(x_train), np.asarray(y_train)

def create_window_df_nn_test(list_dfs, lookback, lookforward, cols_to_use, col_to_pred, scaler_cols):
    x_train, y_train, date_pred, date_last_data = [], [], [], []

    for df_ in list_dfs:
        df = df_.copy()
        df.loc[:, cols_to_use] = scaler_cols.transform(df[cols_to_use])

        for i in range(len(df) - lookback - lookforward + 1):
            x_train.append(np.asarray(df.iloc[i:i+lookback][cols_to_use].values))
            y_train.append(np.asarray(df.iloc[i+lookback+lookforward-1][col_to_pred]))
            date_last_data.append(df.iloc[i+lookback-1]["Datetime"])
            date_pred.append(df.iloc[i+lookback+lookforward-1]["Datetime"])

    return np.asarray(x_train), np.asarray(y_train), np.asarray(date_last_data), np.asarray(date_pred)

In [149]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, explained_variance_score, max_error, median_absolute_error
from math import sqrt
import sklearn

def error_nn(y_pred, y_true, index_scaler):
    y_pred = index_scaler.inverse_transform(y_pred.reshape(-1, 1))
    y_true = index_scaler.inverse_transform(y_true.reshape(-1, 1))

    print(y_pred.reshape(-1).shape)
    print(y_true.reshape(-1).shape)
    df_ = pd.DataFrame({"y_pred": y_pred.reshape(-1), "y_true": y_true.reshape(-1)})

    mse = mean_squared_error(y_true, y_pred)
    rmse = sqrt(mse)
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    # Median Absolute Error
    medae = median_absolute_error(y_true, y_pred)
    # Explained Variance Score
    explained_variance = explained_variance_score(y_true, y_pred)
    # Max Error
    max_err = max_error(y_true, y_pred)


    data = {
        'Métrica': ['RMSE', 'MSE', 'MAE', 'R²', 'MedAE', 'Varianza explicada', 'Max error'],
        'Valor': [rmse, mse, mae, r2, medae, explained_variance, max_err]
    }
    df = pd.DataFrame(data)
    df_invertido = df.transpose()
    df_invertido.columns = df_invertido.iloc[0]
    df_invertido = df_invertido[1:]
    display(df_invertido)

    # scatter plot
    plt.figure(figsize=(8, 4))
    plt.scatter(y_true, y_pred, c='blue', label='Predicciones vs. Valores reales')
    plt.plot([min(y_true), max(y_true)], [min(y_true), max(y_true)], 'k--', lw=2, label='Línea de referencia')
    plt.xlabel('Valores reales')
    plt.ylabel('Predicciones')
    plt.title('Diagrama de dispersión de Predicciones vs. Valores reales')
    plt.legend()
    plt.grid(True)
    plt.show()

    return

# 1. Carga y preparación de datos

## 1.1 Carga de los datos

In [150]:
hd = pd.read_csv(tfm_path+H1_code+'.csv', parse_dates=["Datetime"])
# md = pd.read_csv(tfm_path+M5_code+'.csv', parse_dates=["Datetime"]) # no se va a usar por ahora
historical_storms = pd.read_csv(tfm_path+hstorms_data)
# historical_storms = historical_storms.drop(columns=['Min. Dst (nT)','Unnamed: 0'], axis=1)

- Ver qué columnas y tipo de datos contienen los df.

- Ordenar las tormentas en caso de que no lo estén
- Convertir todas las fechas a datetime de pd.

In [151]:
# Ordenar los dataframes por fecha
historical_storms = historical_storms.sort_values(by='start')

# convertir las columnas de tiempo a datetime64
hd['Datetime']=pd.to_datetime(hd['Datetime'])
historical_storms['start']=pd.to_datetime(historical_storms['start'])
historical_storms['end']=pd.to_datetime(historical_storms['end'])

# Cuando se utilizan los datos a 5 minutos, se unen a 5min
#data = pd.merge(md, hd[["Datetime", "Dst"]], on='Datetime', how='left')

In [152]:
hd = imputar_nan(hd)

<ipython-input-145-15b2522bf30c>:3: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='ffill', inplace=True)
<ipython-input-145-15b2522bf30c>:4: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='bfill', inplace=True)


In [153]:
all_storms = filter_storms(hd, historical_storms, temporal_margin)
all_storms = combinar_dataframes_solapados(all_storms)

In [154]:
historical_storms_new = get_new_splits_dates(all_storms)

Storm index de cada conjunto de datos, test y validación, entrenamiento el resto

In [155]:
index_test_storms = [2, 12, 23, 32, 8, 40, 42, 3, 31]
index_val_storms = [25, 13, 1, 37, 33, 7]

TORMENTAS DE ENTRENAMIENTO

In [156]:
train_storms = [df for idx, df in enumerate(all_storms) if idx not in [x-1 for x in index_test_storms]]
# historical_storms_new[~historical_storms_new["storm_index"].isin(index_test_storms)].reset_index(drop=True)

TORMENTAS DE TEST

In [157]:
train_storms

[      ID_IMF  ID_plasma  Bmag  dev_Bmag   Bx  By_gse  Bz_gse  By_gsm  Bz_gsm  \
 1765    71.0       71.0   4.2       0.2  0.8    -4.0    -0.2    -3.8    -1.2   
 1766    71.0       71.0   4.0       0.1  1.1    -3.8     0.3    -3.7    -0.8   
 1767    71.0       71.0   3.5       0.5 -0.8    -2.9     0.3    -2.8    -0.7   
 1768    71.0       71.0   4.0       0.4 -1.2    -3.2     1.8    -3.6     0.5   
 1769    71.0       71.0   4.4       0.7 -0.7    -1.4     3.4    -2.7     2.5   
 ...      ...        ...   ...       ...  ...     ...     ...     ...     ...   
 2000    71.0       71.0   4.9       0.3 -0.2    -4.7    -0.4    -4.4    -1.6   
 2001    71.0       71.0   5.1       0.5 -0.2    -4.9     0.3    -4.8    -0.9   
 2002    71.0       71.0   4.9       0.2 -0.3    -4.6     0.0    -4.5    -1.0   
 2003    71.0       71.0   4.8       0.3 -2.6    -3.1    -0.4    -2.9    -1.1   
 2004    71.0       71.0   3.9       0.6 -2.7    -2.0    -0.2    -1.9    -0.7   
 
       dev_Bx  ...  P_dens

In [158]:
test_storms = [df for idx, df in enumerate(all_storms) if idx in [x-1 for x in index_test_storms]]
# historical_storms_new[historical_storms_new["storm_index"].isin(index_test_storms)].reset_index(drop=True)

In [159]:
# Función para escalar los datos
def get_scalers(df, cols_to_use, col_to_pred):
    scaler_cols = StandardScaler()
    scaler_label = StandardScaler()
    scaler_cols.fit(df[cols_to_use])
    scaler_label.fit(df[col_to_pred].values.reshape(-1, 1))
    return scaler_cols, scaler_label

# Función para crear ventanas de datos
def create_window_df_nn(df, lookback, lookforward, cols_to_use, col_to_pred, scaler_cols):
    x_train, y_train = [], []
    df_ = df[cols_to_use].copy()
    df_[cols_to_use] = scaler_cols.transform(df_[cols_to_use])
    for i in range(len(df_) - lookback - lookforward + 1):
        x_train.append(df_.iloc[i:i+lookback].values.flatten())
        y_train.append(df_.iloc[i+lookback+lookforward-1][col_to_pred])
    return np.array(x_train), np.array(y_train)

## Modelo optimo a 2h

In [160]:
lookforward = 4
kernel="linear"
c_value=0.7196856730011522
epsilon=0.1

In [161]:
df = pd.concat(train_storms, ignore_index=False)
print(df.shape)
# Obtener los escaladores
scaler_cols, scaler_label = get_scalers(df, cols_to_use, col_to_pred)

# Crear las ventanas de datos
X, y = create_window_df_nn(df, lookback, lookforward, cols_to_use, col_to_pred, scaler_cols)

# Escalar la variable objetivo
y = scaler_label.transform(y.reshape(-1, 1)).flatten()

# Dividir los datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entrenar el modelo
model = SVR(kernel=kernel, C=c_value, epsilon=epsilon)
# # Entrenar el modelo SVR con kernel lineal
# # model = SVR(kernel="linear", C=0.13894954943731375, epsilon=0.1)
# # model = SVR(kernel="linear", C=0.02682695795279726, epsilon=0.1)

model.fit(X_train, y_train)

# Obtener los vectores de soporte, los coeficientes duales y el intercepto
support_vectors = model.support_vectors_
dual_coef = model.dual_coef_[0]
intercept = model.intercept_[0]

# Calcular los coeficientes del modelo (esto es válido solo para kernel lineal)
coefficients = np.dot(dual_coef, support_vectors)

# Construir la ecuación del modelo usando los nombres de las columnas
equation_terms = [f"({coeff:.4f} * {var})" for coeff, var in zip(coefficients, cols_to_use)]
equation = " + ".join(equation_terms) + f" + {intercept:.4f}"

print("Ecuación del modelo:", equation)

# Función para desescalar los coeficientes y el intercepto
def desescalar_modelo(coefficients, intercept, scaler_cols, scaler_label, lookback):
    scale_factors = np.tile(scaler_cols.scale_, lookback)
    mean_factors = np.tile(scaler_cols.mean_, lookback)

    scaled_coefficients = coefficients / scale_factors
    descaled_intercept = intercept - np.sum(scaled_coefficients * mean_factors)
    descaled_intercept = descaled_intercept * scaler_label.scale_ + scaler_label.mean_
    return scaled_coefficients, descaled_intercept

# Datos de entrada de ejemplo
storm_data_test1 = test_storms[4].iloc[130]

# Crear un DataFrame para storm_data_test1 y escalar los datos
storm_data_test1_df = pd.DataFrame([storm_data_test1[cols_to_use].values], columns=cols_to_use)
scaled_storm_data = scaler_cols.transform(storm_data_test1_df)

# Expandir los datos de entrada para que coincidan con la ventana de tiempo
expanded_data = np.tile(scaled_storm_data.flatten(), lookback).reshape(1, -1)

# Calculamos la predicción desescalada
coefficients_descaled, intercept_descaled = desescalar_modelo(coefficients, intercept, scaler_cols, scaler_label, lookback)
prediction_scaled = np.dot(coefficients_descaled, expanded_data.flatten()) + intercept_descaled[0]

# Desescalamos la predicción
prediction = scaler_label.inverse_transform(prediction_scaled.reshape(-1, 1)).flatten()[0]

print(f"Predicción desescalada: {prediction:.4f}")

# Construcción de la ecuación del modelo desescalado
equation_terms_descaled = [f"({coeff} * {var}_{i})" for i in range(lookback) for coeff, var in zip(coefficients_descaled[i*len(cols_to_use):(i+1)*len(cols_to_use)], cols_to_use)]
equation_descaled = " + ".join(equation_terms_descaled) + f" + {intercept_descaled[0]}"

print("Ecuación del modelo desescalado:", equation_descaled)

(8274, 22)
Ecuación del modelo: (0.0002 * Bx) + (0.0002 * By_gse) + (0.0001 * Bz_gse) + (0.0002 * By_gsm) + (0.0003 * Bz_gsm) + (0.0000 * P_density) + (-0.0003 * E_field) + (-0.0002 * plasma_T) + (-0.0002 * plasma_V) + (0.0001 * Dst) + 0.8485
Predicción desescalada: -54.7460
Ecuación del modelo desescalado: (4.778245781711261e-05 * Bx_0) + (3.623862527776346e-05 * By_gse_0) + (2.2794869355402386e-05 * Bz_gse_0) + (3.630989067390762e-05 * By_gsm_0) + (5.9613918475801884e-05 * Bz_gsm_0) + (7.872055057468569e-08 * P_density_0) + (-0.00011013538745152116 * E_field_0) + (-1.255080206422241e-09 * plasma_T_0) + (-1.5900777090843926e-06 * plasma_V_0) + (3.686314652265957e-06 * Dst_0) + (3.447931185320565e-05 * Bx_1) + (7.176394431507544e-06 * By_gse_1) + (4.120002491229751e-05 * Bz_gse_1) + (7.5165478578747345e-06 * By_gsm_1) + (7.748099618600785e-05 * Bz_gsm_1) + (4.996775100391737e-05 * P_density_1) + (-0.00013866600653796588 * E_field_1) + (-9.781809463008189e-10 * plasma_T_1) + (-1.6734410

In [163]:
print("Coef Var")
[print(f"{coeff:.2e}", f"{var}_{i}") for i in range(lookback) for coeff, var in zip(coefficients_descaled[i*len(cols_to_use):(i+1)*len(cols_to_use)], cols_to_use)]

In [164]:
# oeficientes y el intercepto
def desescalar_modelo(coefficients, intercept, scaler_cols, scaler_label, selected_indices):
    scale_factors = scaler_cols.scale_[selected_indices]
    mean_factors = scaler_cols.mean_[selected_indices]

    scaled_coefficients = coefficients / scale_factors
    descaled_intercept = intercept - np.sum(scaled_coefficients * mean_factors)
    descaled_intercept = descaled_intercept * scaler_label.scale_ + scaler_label.mean_
    return scaled_coefficients, descaled_intercept

# Datos de entrenamiento y prueba
df = pd.concat(train_storms, ignore_index=False)
scaler_cols, scaler_label = get_scalers(df, cols_to_use, col_to_pred)
X, y = create_window_df_nn(df, lookback, 1, cols_to_use, col_to_pred, scaler_cols)
y = scaler_label.transform(y.reshape(-1, 1)).flatten()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entrenar el modelo utilizando RFECV
svr = SVR(kernel="linear", C=0.7196856730011522, epsilon=0.1)
rfecv = RFECV(estimator=svr, step=1, cv=5, scoring='neg_mean_squared_error')
rfecv.fit(X_train, y_train)

# Obtener los índices de las características seleccionadas
selected_features = rfecv.support_
selected_indices = np.where(selected_features)[0]
selected_cols = [f"{cols_to_use[i % len(cols_to_use)]}_{i // len(cols_to_use)}" for i in selected_indices]

# Entrenar el modelo final con las características seleccionadas
X_train_selected = rfecv.transform(X_train)
X_test_selected = rfecv.transform(X_test)
svr.fit(X_train_selected, y_train)

# Obtener los coeficientes y el intercepto del modelo
coefficients = svr.coef_.flatten()
intercept = svr.intercept_

# Desescalar los coeficientes y el intercepto
coefficients_descaled, intercept_descaled = desescalar_modelo(coefficients, intercept, scaler_cols, scaler_label, selected_indices % len(cols_to_use))

# Construir la ecuación desescalada
equation_terms_descaled = ["({:.2e} \\cdot \\textnormal{{{}}})".format(coeff, var.replace("_", r"\_")) for coeff, var in zip(coefficients_descaled, selected_cols)]
equation_descaled = " + ".join(equation_terms_descaled) + " + {:.4f}".format(intercept_descaled[0])

print("Ecuación del modelo desescalado:", equation_descaled)

# Predicción con las características seleccionadas
storm_data_test1 = test_storms[4].iloc[130]
storm_data_test1_df = pd.DataFrame([storm_data_test1[cols_to_use].values], columns=cols_to_use)
scaled_storm_data = scaler_cols.transform(storm_data_test1_df)

# Expandir y seleccionar las características relevantes para la predicción
expanded_data = np.tile(scaled_storm_data.flatten(), lookback).reshape(1, -1)
expanded_data_selected = rfecv.transform(expanded_data)

# Calculamos la predicción desescalada
prediction_scaled = svr.predict(expanded_data_selected)
prediction = scaler_label.inverse_transform(prediction_scaled.reshape(-1, 1)).flatten()[0]

print(f"Predicción desescalada: {prediction:.4f}")

Ecuación del modelo desescalado: (-8.86e-04 \cdot \textnormal{E\_field\_8}) + (5.77e-04 \cdot \textnormal{Bz\_gse\_9}) + (-1.20e-03 \cdot \textnormal{E\_field\_9}) + (6.90e-05 \cdot \textnormal{Dst\_9}) + (7.35e-05 \cdot \textnormal{Dst\_10}) + (1.03e-04 \cdot \textnormal{Dst\_11}) + -0.9870
Predicción desescalada: -2.0379


## Modelo optimo a 4h

In [165]:
lookforward = 4
kernel="linear"
c_value=0.13894954943731375
epsilon=0.1

In [166]:
df = pd.concat(train_storms, ignore_index=False)
print(df.shape)
# Obtener los escaladores
scaler_cols, scaler_label = get_scalers(df, cols_to_use, col_to_pred)

# Crear las ventanas de datos
X, y = create_window_df_nn(df, lookback, lookforward, cols_to_use, col_to_pred, scaler_cols)

# Escalar la variable objetivo
y = scaler_label.transform(y.reshape(-1, 1)).flatten()

# Dividir los datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entrenar el modelo
model = SVR(kernel=kernel, C=c_value, epsilon=epsilon)
# # Entrenar el modelo SVR con kernel lineal
# # model = SVR(kernel="linear", C=0.13894954943731375, epsilon=0.1)
# # model = SVR(kernel="linear", C=0.02682695795279726, epsilon=0.1)

model.fit(X_train, y_train)

# Obtener los vectores de soporte, los coeficientes duales y el intercepto
support_vectors = model.support_vectors_
dual_coef = model.dual_coef_[0]
intercept = model.intercept_[0]

# Calcular los coeficientes del modelo (esto es válido solo para kernel lineal)
coefficients = np.dot(dual_coef, support_vectors)

# Construir la ecuación del modelo usando los nombres de las columnas
equation_terms = [f"({coeff:.4f} * {var})" for coeff, var in zip(coefficients, cols_to_use)]
equation = " + ".join(equation_terms) + f" + {intercept:.4f}"

print("Ecuación del modelo:", equation)

# Función para desescalar los coeficientes y el intercepto
def desescalar_modelo(coefficients, intercept, scaler_cols, scaler_label, lookback):
    scale_factors = np.tile(scaler_cols.scale_, lookback)
    mean_factors = np.tile(scaler_cols.mean_, lookback)

    scaled_coefficients = coefficients / scale_factors
    descaled_intercept = intercept - np.sum(scaled_coefficients * mean_factors)
    descaled_intercept = descaled_intercept * scaler_label.scale_ + scaler_label.mean_
    return scaled_coefficients, descaled_intercept

# Datos de entrada de ejemplo
storm_data_test1 = test_storms[4].iloc[130]

# Crear un DataFrame para storm_data_test1 y escalar los datos
storm_data_test1_df = pd.DataFrame([storm_data_test1[cols_to_use].values], columns=cols_to_use)
scaled_storm_data = scaler_cols.transform(storm_data_test1_df)

# Expandir los datos de entrada para que coincidan con la ventana de tiempo
expanded_data = np.tile(scaled_storm_data.flatten(), lookback).reshape(1, -1)

# Calculamos la predicción desescalada
coefficients_descaled, intercept_descaled = desescalar_modelo(coefficients, intercept, scaler_cols, scaler_label, lookback)
prediction_scaled = np.dot(coefficients_descaled, expanded_data.flatten()) + intercept_descaled[0]

# Desescalamos la predicción
prediction = scaler_label.inverse_transform(prediction_scaled.reshape(-1, 1)).flatten()[0]

print(f"Predicción desescalada: {prediction:.4f}")

# Construcción de la ecuación del modelo desescalado
equation_terms_descaled = [f"({coeff} * {var}_{i})" for i in range(lookback) for coeff, var in zip(coefficients_descaled[i*len(cols_to_use):(i+1)*len(cols_to_use)], cols_to_use)]
equation_descaled = " + ".join(equation_terms_descaled) + f" + {intercept_descaled[0]}"

print("Ecuación del modelo desescalado:", equation_descaled)

(8274, 22)
Ecuación del modelo: (0.0002 * Bx) + (0.0002 * By_gse) + (0.0001 * Bz_gse) + (0.0002 * By_gsm) + (0.0003 * Bz_gsm) + (0.0000 * P_density) + (-0.0003 * E_field) + (-0.0002 * plasma_T) + (-0.0002 * plasma_V) + (0.0001 * Dst) + 0.8485
Predicción desescalada: -54.7460
Ecuación del modelo desescalado: (4.778245781711261e-05 * Bx_0) + (3.623862527776346e-05 * By_gse_0) + (2.2794869355402386e-05 * Bz_gse_0) + (3.630989067390762e-05 * By_gsm_0) + (5.9613918475801884e-05 * Bz_gsm_0) + (7.872055057468569e-08 * P_density_0) + (-0.00011013538745152116 * E_field_0) + (-1.255080206422241e-09 * plasma_T_0) + (-1.5900777090843926e-06 * plasma_V_0) + (3.686314652265957e-06 * Dst_0) + (3.447931185320565e-05 * Bx_1) + (7.176394431507544e-06 * By_gse_1) + (4.120002491229751e-05 * Bz_gse_1) + (7.5165478578747345e-06 * By_gsm_1) + (7.748099618600785e-05 * Bz_gsm_1) + (4.996775100391737e-05 * P_density_1) + (-0.00013866600653796588 * E_field_1) + (-9.781809463008189e-10 * plasma_T_1) + (-1.6734410

In [168]:
print("Coef Var")
[print(f"{coeff:.2e}", f"{var}_{i}") for i in range(lookback) for coeff, var in zip(coefficients_descaled[i*len(cols_to_use):(i+1)*len(cols_to_use)], cols_to_use)]

In [169]:
# oeficientes y el intercepto
def desescalar_modelo(coefficients, intercept, scaler_cols, scaler_label, selected_indices):
    scale_factors = scaler_cols.scale_[selected_indices]
    mean_factors = scaler_cols.mean_[selected_indices]

    scaled_coefficients = coefficients / scale_factors
    descaled_intercept = intercept - np.sum(scaled_coefficients * mean_factors)
    descaled_intercept = descaled_intercept * scaler_label.scale_ + scaler_label.mean_
    return scaled_coefficients, descaled_intercept

# Datos de entrenamiento y prueba
df = pd.concat(train_storms, ignore_index=False)
scaler_cols, scaler_label = get_scalers(df, cols_to_use, col_to_pred)
X, y = create_window_df_nn(df, lookback, 1, cols_to_use, col_to_pred, scaler_cols)
y = scaler_label.transform(y.reshape(-1, 1)).flatten()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entrenar el modelo utilizando RFECV
svr = SVR(kernel="linear", C=0.7196856730011522, epsilon=0.1)
rfecv = RFECV(estimator=svr, step=1, cv=5, scoring='neg_mean_squared_error')
rfecv.fit(X_train, y_train)

# Obtener los índices de las características seleccionadas
selected_features = rfecv.support_
selected_indices = np.where(selected_features)[0]
selected_cols = [f"{cols_to_use[i % len(cols_to_use)]}_{i // len(cols_to_use)}" for i in selected_indices]

# Entrenar el modelo final con las características seleccionadas
X_train_selected = rfecv.transform(X_train)
X_test_selected = rfecv.transform(X_test)
svr.fit(X_train_selected, y_train)

# Obtener los coeficientes y el intercepto del modelo
coefficients = svr.coef_.flatten()
intercept = svr.intercept_

# Desescalar los coeficientes y el intercepto
coefficients_descaled, intercept_descaled = desescalar_modelo(coefficients, intercept, scaler_cols, scaler_label, selected_indices % len(cols_to_use))

# Construir la ecuación desescalada
equation_terms_descaled = ["({:.2e} \\cdot \\textnormal{{{}}})".format(coeff, var.replace("_", r"\_")) for coeff, var in zip(coefficients_descaled, selected_cols)]
equation_descaled = " + ".join(equation_terms_descaled) + " + {:.4f}".format(intercept_descaled[0])

print("Ecuación del modelo desescalado:", equation_descaled)

# Predicción con las características seleccionadas
storm_data_test1 = test_storms[4].iloc[130]
storm_data_test1_df = pd.DataFrame([storm_data_test1[cols_to_use].values], columns=cols_to_use)
scaled_storm_data = scaler_cols.transform(storm_data_test1_df)

# Expandir y seleccionar las características relevantes para la predicción
expanded_data = np.tile(scaled_storm_data.flatten(), lookback).reshape(1, -1)
expanded_data_selected = rfecv.transform(expanded_data)

# Calculamos la predicción desescalada
prediction_scaled = svr.predict(expanded_data_selected)
prediction = scaler_label.inverse_transform(prediction_scaled.reshape(-1, 1)).flatten()[0]

print(f"Predicción desescalada: {prediction:.4f}")

Ecuación del modelo desescalado: (-8.86e-04 \cdot \textnormal{E\_field\_8}) + (5.77e-04 \cdot \textnormal{Bz\_gse\_9}) + (-1.20e-03 \cdot \textnormal{E\_field\_9}) + (6.90e-05 \cdot \textnormal{Dst\_9}) + (7.35e-05 \cdot \textnormal{Dst\_10}) + (1.03e-04 \cdot \textnormal{Dst\_11}) + -0.9870
Predicción desescalada: -2.0379


## Modelo optimo a 6h

In [176]:
lookforward = 4
kernel="linear"
c_value=0.02682695795279726
epsilon=0.1

In [177]:
df = pd.concat(train_storms, ignore_index=False)
print(df.shape)
# Obtener los escaladores
scaler_cols, scaler_label = get_scalers(df, cols_to_use, col_to_pred)

# Crear las ventanas de datos
X, y = create_window_df_nn(df, lookback, lookforward, cols_to_use, col_to_pred, scaler_cols)

# Escalar la variable objetivo
y = scaler_label.transform(y.reshape(-1, 1)).flatten()

# Dividir los datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entrenar el modelo
model = SVR(kernel=kernel, C=c_value, epsilon=epsilon)
# # Entrenar el modelo SVR con kernel lineal
# # model = SVR(kernel="linear", C=0.13894954943731375, epsilon=0.1)
# # model = SVR(kernel="linear", C=0.02682695795279726, epsilon=0.1)

model.fit(X_train, y_train)

# Obtener los vectores de soporte, los coeficientes duales y el intercepto
support_vectors = model.support_vectors_
dual_coef = model.dual_coef_[0]
intercept = model.intercept_[0]

# Calcular los coeficientes del modelo (esto es válido solo para kernel lineal)
coefficients = np.dot(dual_coef, support_vectors)

# Construir la ecuación del modelo usando los nombres de las columnas
equation_terms = [f"({coeff:.4f} * {var})" for coeff, var in zip(coefficients, cols_to_use)]
equation = " + ".join(equation_terms) + f" + {intercept:.4f}"

print("Ecuación del modelo:", equation)

# Función para desescalar los coeficientes y el intercepto
def desescalar_modelo(coefficients, intercept, scaler_cols, scaler_label, lookback):
    scale_factors = np.tile(scaler_cols.scale_, lookback)
    mean_factors = np.tile(scaler_cols.mean_, lookback)

    scaled_coefficients = coefficients / scale_factors
    descaled_intercept = intercept - np.sum(scaled_coefficients * mean_factors)
    descaled_intercept = descaled_intercept * scaler_label.scale_ + scaler_label.mean_
    return scaled_coefficients, descaled_intercept

# Datos de entrada de ejemplo
storm_data_test1 = test_storms[4].iloc[130]

# Crear un DataFrame para storm_data_test1 y escalar los datos
storm_data_test1_df = pd.DataFrame([storm_data_test1[cols_to_use].values], columns=cols_to_use)
scaled_storm_data = scaler_cols.transform(storm_data_test1_df)

# Expandir los datos de entrada para que coincidan con la ventana de tiempo
expanded_data = np.tile(scaled_storm_data.flatten(), lookback).reshape(1, -1)

# Calculamos la predicción desescalada
coefficients_descaled, intercept_descaled = desescalar_modelo(coefficients, intercept, scaler_cols, scaler_label, lookback)
prediction_scaled = np.dot(coefficients_descaled, expanded_data.flatten()) + intercept_descaled[0]

# Desescalamos la predicción
prediction = scaler_label.inverse_transform(prediction_scaled.reshape(-1, 1)).flatten()[0]

print(f"Predicción desescalada: {prediction:.4f}")

# Construcción de la ecuación del modelo desescalado
equation_terms_descaled = [f"({coeff} * {var}_{i})" for i in range(lookback) for coeff, var in zip(coefficients_descaled[i*len(cols_to_use):(i+1)*len(cols_to_use)], cols_to_use)]
equation_descaled = " + ".join(equation_terms_descaled) + f" + {intercept_descaled[0]}"

print("Ecuación del modelo desescalado:", equation_descaled)

(8274, 22)
Ecuación del modelo: (0.0002 * Bx) + (0.0002 * By_gse) + (0.0001 * Bz_gse) + (0.0002 * By_gsm) + (0.0003 * Bz_gsm) + (0.0000 * P_density) + (-0.0003 * E_field) + (-0.0002 * plasma_T) + (-0.0002 * plasma_V) + (0.0001 * Dst) + 0.8485
Predicción desescalada: -54.7460
Ecuación del modelo desescalado: (4.778245781711261e-05 * Bx_0) + (3.623862527776346e-05 * By_gse_0) + (2.2794869355402386e-05 * Bz_gse_0) + (3.630989067390762e-05 * By_gsm_0) + (5.9613918475801884e-05 * Bz_gsm_0) + (7.872055057468569e-08 * P_density_0) + (-0.00011013538745152116 * E_field_0) + (-1.255080206422241e-09 * plasma_T_0) + (-1.5900777090843926e-06 * plasma_V_0) + (3.686314652265957e-06 * Dst_0) + (3.447931185320565e-05 * Bx_1) + (7.176394431507544e-06 * By_gse_1) + (4.120002491229751e-05 * Bz_gse_1) + (7.5165478578747345e-06 * By_gsm_1) + (7.748099618600785e-05 * Bz_gsm_1) + (4.996775100391737e-05 * P_density_1) + (-0.00013866600653796588 * E_field_1) + (-9.781809463008189e-10 * plasma_T_1) + (-1.6734410

In [179]:
print("Coef Var")
[print(f"{coeff:.2e}", f"{var}_{i}") for i in range(lookback) for coeff, var in zip(coefficients_descaled[i*len(cols_to_use):(i+1)*len(cols_to_use)], cols_to_use)]

In [180]:
# oeficientes y el intercepto
def desescalar_modelo(coefficients, intercept, scaler_cols, scaler_label, selected_indices):
    scale_factors = scaler_cols.scale_[selected_indices]
    mean_factors = scaler_cols.mean_[selected_indices]

    scaled_coefficients = coefficients / scale_factors
    descaled_intercept = intercept - np.sum(scaled_coefficients * mean_factors)
    descaled_intercept = descaled_intercept * scaler_label.scale_ + scaler_label.mean_
    return scaled_coefficients, descaled_intercept

# Datos de entrenamiento y prueba
df = pd.concat(train_storms, ignore_index=False)
scaler_cols, scaler_label = get_scalers(df, cols_to_use, col_to_pred)
X, y = create_window_df_nn(df, lookback, 1, cols_to_use, col_to_pred, scaler_cols)
y = scaler_label.transform(y.reshape(-1, 1)).flatten()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entrenar el modelo utilizando RFECV
svr = SVR(kernel="linear", C=0.7196856730011522, epsilon=0.1)
rfecv = RFECV(estimator=svr, step=1, cv=5, scoring='neg_mean_squared_error')
rfecv.fit(X_train, y_train)

# Obtener los índices de las características seleccionadas
selected_features = rfecv.support_
selected_indices = np.where(selected_features)[0]
selected_cols = [f"{cols_to_use[i % len(cols_to_use)]}_{i // len(cols_to_use)}" for i in selected_indices]

# Entrenar el modelo final con las características seleccionadas
X_train_selected = rfecv.transform(X_train)
X_test_selected = rfecv.transform(X_test)
svr.fit(X_train_selected, y_train)

# Obtener los coeficientes y el intercepto del modelo
coefficients = svr.coef_.flatten()
intercept = svr.intercept_

# Desescalar los coeficientes y el intercepto
coefficients_descaled, intercept_descaled = desescalar_modelo(coefficients, intercept, scaler_cols, scaler_label, selected_indices % len(cols_to_use))

# Construir la ecuación desescalada
equation_terms_descaled = ["({:.2e} \\cdot \\textnormal{{{}}})".format(coeff, var.replace("_", r"\_")) for coeff, var in zip(coefficients_descaled, selected_cols)]
equation_descaled = " + ".join(equation_terms_descaled) + " + {:.4f}".format(intercept_descaled[0])

print("Ecuación del modelo desescalado:", equation_descaled)

# Predicción con las características seleccionadas
storm_data_test1 = test_storms[4].iloc[130]
storm_data_test1_df = pd.DataFrame([storm_data_test1[cols_to_use].values], columns=cols_to_use)
scaled_storm_data = scaler_cols.transform(storm_data_test1_df)

# Expandir y seleccionar las características relevantes para la predicción
expanded_data = np.tile(scaled_storm_data.flatten(), lookback).reshape(1, -1)
expanded_data_selected = rfecv.transform(expanded_data)

# Calculamos la predicción desescalada
prediction_scaled = svr.predict(expanded_data_selected)
prediction = scaler_label.inverse_transform(prediction_scaled.reshape(-1, 1)).flatten()[0]

print(f"Predicción desescalada: {prediction:.4f}")

Ecuación del modelo desescalado: (-8.86e-04 \cdot \textnormal{E\_field\_8}) + (5.77e-04 \cdot \textnormal{Bz\_gse\_9}) + (-1.20e-03 \cdot \textnormal{E\_field\_9}) + (6.90e-05 \cdot \textnormal{Dst\_9}) + (7.35e-05 \cdot \textnormal{Dst\_10}) + (1.03e-04 \cdot \textnormal{Dst\_11}) + -0.9870
Predicción desescalada: -2.0379
